In [1]:
import os
import torch
import logging
import argparse
import math
import numpy as np
from tqdm import tqdm
import multiprocessing
import time
import json
from torch.utils.data import Dataset
from itertools import cycle
from torch.utils.data import DataLoader, RandomSampler
from torch.utils.data.distributed import DistributedSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from models import build_or_load_gen_model
from configs import add_args, set_seed, set_dist
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import os
from transformers import T5Tokenizer, T5Model
from utils import read_review_examples, TextDataset, CommentGenDataset, CommentClsDataset
from models import build_or_load_gen_model
from run_finetune_cls import get_loaders
import multiprocessing
import torch.distributed as dist

In [20]:
mnt_dir = "/home/v-zhuoli1/lzzz"
class FakeArgs(object):
    def __init__(self):
        self.max_source_length = 512
        self.max_target_length = 128
        # self.model_type = "scratch"
        self.model_type = "codet5"
        self.config_name = f"{mnt_dir}/PreViewer/saved_models_codet5_shuai/save_codet5/checkpoints-245000-3.97"
        self.add_lang_ids = True
        # self.tokenizer_path = "tokenizer"
        self.tokenizer_path = f"{mnt_dir}/PreViewer/pretrained_models/codet5"
        self.model_name_or_path = self.config_name
        self.load_model_path = None
        # self.train_path = "../../../lzzz/processed"
        self.train_batch_size = 1
        self.cpu_count = 3
        self.num_train_epochs = 2
        self.gradient_accumulation_steps = 1
        self.mask_rate = 0.15
        self.from_scratch = False
        self.beam_size = 6
        self.local_rank = 0
        self.global_rank = 0
        self.world_size = 1

args = FakeArgs()
config, model, tokenizer = build_or_load_gen_model(args)
model.to(args.local_rank)

02/24/2022 06:52:56 - INFO - models -   Finish loading model [223M] from /home/v-zhuoli1/lzzz/PreViewer/saved_models_codet5_shuai/save_codet5/checkpoints-245000-3.97


ReviewerModel(
  (shared): Embedding(32216, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32216, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dropout(p=0.1, 

In [6]:
def generate_from_ids(ids, args, model, tokenizer):
    pred_ids = []
    source_ids = torch.tensor(
        [ids], dtype=torch.long
    ).to(args.local_rank)
    source_mask = source_ids.ne(tokenizer.pad_id)
    preds = model.generate(source_ids,
                        attention_mask=source_mask,
                        use_cache=True,
                        num_beams=args.beam_size,
                        early_stopping=True,
                        max_length=args.max_target_length)
    top_preds = list(preds.cpu().numpy())
    pred_ids.extend(top_preds)
    pred_nls = [tokenizer.decode(id, skip_special_tokens=True, clean_up_tokenization_spaces=False) for id in pred_ids]
    return pred_nls

In [17]:
model.cls_head = None
model.load_state_dict(torch.load(os.path.join(args.config_name, "pytorch_model.bin")))

<All keys matched successfully>

In [21]:
tokens = ['<s>', 'this', '.', 't', 'err', 'ia', '.', 'selected', 'Feature', '\u0120=', '\u0120this', '.', 'feature', ';', '}', '//', '\u0120ensure', '\u0120the', '\u0120targeting', '\u0120cursor', '\u0120keeps', '\u0120updating', '\u0120(', 'as', '\u0120it', '\u0120is', '\u0120hook', 'ed', '\u0120into', '\u0120the', '\u0120C', 'es', 'ium', '\u0120render', '\u0120loop', ')', 'this', '.', 't', 'err', 'ia', '.', 'current', 'Viewer', '.', 'notify', 'Rep', 'aint', 'Required', '();', '};', 'function', '\u0120html', 'From', 'Feature', '(', 'view', 'Model', ',', '\u0120clock', ')', '\u0120{', '//', '\u0120If', '\u0120a', '\u0120template', '\u0120is', '\u0120defined', ',', '\u0120render', '\u0120it', '\u0120using', '\u0120feature', '.', 'properties', ',', '\u0120which', '\u0120is', '\u0120non', '-', 'time', '-', 'v', 'ary', 'ing', '.', '//', '\u0120If', '\u0120no', '\u0120template', '\u0120is', '\u0120provided', ',', '\u0120show', '\u0120feature', '.', 'description', ',', '\u0120which', '\u0120may', '\u0120be', '\u0120time', '-', 'v', 'ary', 'ing', '.', '<start>', '<keep>', 'var', '\u0120feature', '\u0120=', '\u0120viewModel', '.', 'feature', ';', '<keep>', 'var', '\u0120data', '\u0120=', '\u0120viewModel', '.', 'feature', '.', 'properties', ';', '<keep>', 'if', '\u0120(', 'defined', '(', 'view', 'Model', '.', 'template', '))', '\u0120{', '<del>', 'return', '\u0120read', 'Template', '(', 'data', ',', '\u0120viewModel', '.', 'template', ');', '<add>', '//', '\u0120template', '\u0120may', '\u0120be', '\u0120a', '\u0120string', ',', '\u0120eg', '.', "\u0120'<", 'div', '>', '{{', '{', 'F', 'oo', '}}', '}', '\u0120Hello', '\u0120{{', 'name', '}}', '</', 'div', ">'", '<add>', 'if', '\u0120(', 'typeof', '\u0120viewModel', '.', 'template', '\u0120===', "\u0120'", 'string', "')", '\u0120{', '<add>', 'return', '\u0120Must', 'ache', '.', 'render', '(', 'view', 'Model', '.', 'template', ',', '\u0120data', ');', '<add>', '}', '\u0120else', '\u0120{', '<add>', '//', '\u0120or', '\u0120template', '\u0120may', '\u0120be', '\u0120an', '\u0120object', '\u0120with', '\u0120a', '\u0120main', "\u0120'", 'template', "'", '\u0120key', ',', '\u0120and', '\u0120additional', '\u0120partials', '<add>', '//', '\u0120eg', '.', '\u0120{', 'template', ':', "\u0120'<", 'div', '>', 'test', '\u0120{{', '>', 'fo', 'ob', 'ar', '}}', '</', 'div', ">'", ',', '\u0120fo', 'ob', 'ar', ':', "\u0120'<", 'b', '>', '{{', 'F', 'oo', '}}', '</', 'b', ">'", '}', '<add>', 'var', '\u0120template', '\u0120=', '\u0120viewModel', '.', 'template', '.', 'template', ';', '<add>', 'var', '\u0120partials', '\u0120=', '\u0120viewModel', '.', 'template', ';', '<add>', 'delete', '\u0120partials', '.', 'template', ';', '<add>', 'return', '\u0120Must', 'ache', '.', 'render', '(', 'template', ',', '\u0120data', ',', '\u0120partials', ');', '<add>', '}', '<keep>', '}', '<keep>', 'var', '\u0120description', '\u0120=', '\u0120feature', '.', 'description', '.', 'getValue', '(', 'clock', '.', 'current', 'Time', ');', '<keep>', 'if', '\u0120(', 'description', '.', 'properties', ')', '\u0120{', '<end>', 'return', '\u0120JSON', '.', 'stringify', '(', 'description', '.', 'properties', ');', '}', 'return', '\u0120description', ';', '}', 'function', '\u0120configure', 'Html', 'Updater', '(', 'view', 'Model', ')', '\u0120{', '//', '\u0120When', '\u0120no', '\u0120template', '\u0120is', '\u0120provided', ',', '\u0120and', '\u0120feature', '.', 'description', '\u0120is', '\u0120time', '-', 'v', 'ary', 'ing', ',', '\u0120we', '\u0120need', '\u0120to', '\u0120keep', '\u0120updating', '\u0120the', '\u0120description', 'if', '\u0120(!', 'defined', '(', 'view', 'Model', '.', 'template', ')', '\u0120&&', '\u0120!', 'view', 'Model', '.', 'feature', '.', 'description', '.', 'is', 'Constant', ')', '\u0120{', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<p[1,0]<stderr>:ad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']
# tokens = eval(open("txt.py", "r").read())
ids = tokenizer.convert_tokens_to_ids(tokens)
generate_from_ids(ids, args, model, tokenizer)

[' purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpose purpo

In [6]:
model.encoder.embed_tokens.weight.shape

torch.Size([50221, 768])

In [24]:
gold_src, gold_tgt = [], []
with open(datafiles[0], "r") as f:
    for line in f:
        line = line.strip()
        js = json.loads(line)
        if "msg" not in js or len(js["msg"]) == 0:
            continue
        src = js["patch"].replace("\n", "\\n")
        tgt = js["msg"].replace("\n", " ")
        gold_src.append(src)
        gold_tgt.append(tgt)
len(gold_src)

12795

In [27]:
for i, examples in enumerate(loader):
    if i == 1:
        break
    # source_ids = torch.tensor(
    #     [ex.source_ids for ex in examples], dtype=torch.long
    # )
    # source_labels = torch.tensor(
    #     [ex.source_labels for ex in examples], dtype=torch.long
    # )
    # target_ids = torch.tensor(
    #     [ex.target_ids for ex in examples], dtype=torch.long
    # )
    for ex in examples:
        # if ex.type == "label":
        # print(f"example type: {ex.type}")
        # print(f"batch size: {len(examples)}")
        # print(f"example source: {tokenizer.convert_ids_to_tokens(ex.source_ids)}")
        # print(f"example label: {ex.source_labels}")
        # print(f"example target: {tokenizer.convert_ids_to_tokens(ex.target_ids)}")
        source_str = tokenizer.decode(ex.source_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        target_str = tokenizer.decode(ex.target_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        id = ex.example_id
        print(id)
        open("modelsrc.txt", "w").write(source_str)
        open("modeltgt.txt", "w").write(target_str)
        open("examplesrc.txt", "w").write(gold_src[id].replace("\\n", "\n"))
        open("exampletgt.txt", "w").write(gold_tgt[id])
    # source_mask = source_ids.ne(tokenizer.pattd_id)
    # target_mask = target_ids.ne(tokenizer.pad_id)
    # d = model(
    #     input_ids=source_ids,
    #     input_labels=source_labels,
    #     decoder_input_ids=target_ids,
    #     attention_mask=source_mask,
    #     decoder_attention_mask=target_mask,
    # )
    # print(d)

3691


In [ ]:
ids = tokenizer.encode("testo")
tokenizer.convert_ids_to_tokens(ids)

In [ ]:
inputs = data[0].source_ids
labels = data[0].source_labels
targets = data[0].target_ids
# print("".join())

In [ ]:
def prettyprint(inputs):
    for i, tok in enumerate(inputs):
        if i > 0 and str(tok).startswith("<e"):
            print("")
        print(str(tok) + " ", end="")
toks = tokenizer.convert_ids_to_tokens(inputs)
labs = labels
tgts = tokenizer.convert_ids_to_tokens(targets)
# print(toks)
prettyprint(toks)
print("\n\n\n\n\n")
prettyprint(labs)
print("\n\n\n\n\n")
prettyprint(tgts)
print("\n\n\n\n\n")


In [ ]:
print("\n".join(dataset.examples[0].prevlines))
print("\n\n\n\n\n")
print("\n".join(dataset.examples[0].lines))
print("\n\n\n\n\n")
print("\n".join(dataset.examples[0].afterlines))
print("\n\n\n\n\n")
print(dataset.examples[0].msg)

In [ ]:
import json
data = []
idx = 0
with open("../../../lzzz/processed/ruby_cls.jsonl", "r") as f:
    for line in f:
        js = json.loads(line)
        data.append(js)
        # print(js["patch"])
        idx += 1
        if idx == 100:
            break

In [ ]:
import os, time
import json
from tokenizers import ByteLevelBPETokenizer

In [ ]:
tok.encode("ff f").ids

In [ ]:
tok = ByteLevelBPETokenizer("tokenizer/vocab.json", "tokenizer/merges.txt")
tok.encode("+").ids

In [ ]:
from utils import ReviewExample

In [ ]:
exp = ReviewExample(0, open("oldf.txt").read(), open("diff.txt").read(), "it does not matter.", -1)

In [ ]:
exp.labels[105:120]
exp.msg

In [ ]:
exp.input.split("<e0>")[105:120]

In [ ]:
import os, json
path = "../../../lzzz/processed"
files = os.listdir(path)
files = [os.path.join(path, f) for f in files if f.startswith("genchunk_train") and f.endswith(".jsonl")]
# files = files[:1]
allcnt, cnt, cnt2, cnt3 = 0, 0, 0, 0
for file in files:
    f = open(file, "r")
    for line in f:
        js = json.loads(line)
        if "```suggestion" in js["msg"][:15]:
            # open("oldf.txt", "w").write(js["oldf"])
            # open("diff.txt", "w").write(js["patch"])
            # open("msg.txt", "w").write(js["msg"])
            cnt += 1
        if js["msg"].startswith("```suggestion"):
            cnt2 += 1
        if "suggestion" in js["msg"]:
            cnt3 += 1
        allcnt += 1
    f.close()
print(allcnt, cnt, cnt2, cnt3)

In [ ]:
js = json.loads(f.readline())
open("oldf.txt", "w").write(js["oldf"])
open("diff.txt", "w").write(js["patch"])
open("msg.txt", "w").write(js["msg"])

In [ ]:
js["msg"]

In [8]:
from utils import read_review_examples, TextDataset, MyTokenizer
import multiprocessing
cpu_count = multiprocessing.cpu_count()
pool = multiprocessing.Pool(cpu_count)
from transformers import T5Tokenizer
class FakeArgs:
    def __init__(self):
        self.max_source_length = 512
        self.max_target_length = 256
args = FakeArgs()
# tokenizer = T5Tokenizer.from_pretrained("t5-base")
tokenizer = MyTokenizer.from_pretrained("tokenizer")
class DebugDataset(TextDataset):
    def __init__(self, tokenizer, pool, args, file_path, samplenum=-1):
        self.cnt = 0
        savep = file_path.replace(".jsonl", ".exps")
        examples = read_review_examples(file_path, samplenum)
        self.examples = pool.map(self.tokenize, \
            [(example, tokenizer, args) for example in examples])

In [10]:
examples = read_review_examples(file_path, samplenum)

In [20]:
v = []
for i, ex in enumerate(examples):
    line = ex.input
    f = tokenizer.encode(line)
    l1 = len(f) - 1
    lines = line.split("<e0>")
    lines = [l.split() for l in lines]
    l2 = sum(map(len, lines))
    v.append(l1 / l2)
print(sum(v) / len(v))

2.871458557455923


In [ ]:
tokenizer.add_special_tokens({"": ["<e0>"]})
tokenizer.special_dict = {"</s>": tokenizer.get_vocab()["<e0>"]}
file_path = "/home/v-zhuoli1/lzzz/processed/chunk_25.jsonl"
samplenum = 1000
d = DebugDataset(tokenizer, pool, args, file_path, samplenum)

In [ ]:
for i, ex in enumerate(d.examples):
    labels = ex.labels
    if sum(labels) == len(labels) * -100:
        print(i, ex.msg)
        print(ex.lines)
        print(ex.labels)
        # break
print(len(d.examples))

In [2]:
import json
path = "/home/v-zhuoli1/lzzz/processed/genchunk_train0.jsonl"
msgs = []
with open(path, "r") as f:
    for line in f:
        js = json.loads(line)
        msgs.append(js["msg"])
print(f"Example nums: {len(msgs)}")

Example nums: 101937


In [3]:
import random
smsgs = random.sample(msgs, 200)

In [6]:
smsgs = [s.replace("\n", " ") for s in smsgs]
smsgs

['just noticed there are two whitespaces after the `?` :smile: ',
 "We chatted a bit about this in DM's, would you mind summarizing the result of our conversation (RE: why this is fine to remove)?",
 "Can I suggest you to add more debug logs on key places? It was really hard for me to find cassandra.ensureSchema was really set to false, I thought it could be an error to set the variable... I think it's worth logging it as well, as debug  Not on this PR, but I think it would be great as well to have a command line option on zipkin server to dump all spring config. It's not something hard to do, but extremely helpful for the user.",
 "I skimmed this and didn't immediately see the syntax error, can you just add a comment explaining it? Thanks!",
 "`dask_array_type` is actually a tuple, so this needs to be `(Iterable,) + dask_array_type`.  That said... this already tricky logic is getting even more convoluted. So this isn't my favorite fix, though I agree that an N-dimensional dask array s

In [1]:
import json

In [4]:
dataf = "../../../lzzz/processed/python_gen.jsonl"
dics = []
with open(dataf, "r") as f:
    for line in f:
        js = json.loads(line)
        dics.append(js)

In [7]:
def countpro(s, dics):
    cnt = 0
    for dic in dics:
        if s in dic["project"]:
            cnt += 1
    print(cnt)
countpro("pandas-dev-pandas", dics)

3441


In [5]:
import os, json
import random

dataf = "../../../lzzz/reviewdata"
files = os.listdir(dataf)
cls_files = [os.path.join(dataf, f) for f in files if f.startswith("review_cls") and f.endswith(".jsonl")]
gen_files = [os.path.join(dataf, f) for f in files if f.startswith("review_gen") and f.endswith(".jsonl")]
cls_file = random.choice(cls_files)
gen_file = random.choice(gen_files)

def read_jsonl(file):
    dics = []
    with open(file, "r") as f:
        for line in f:
            js = json.loads(line)
            dics.append(js)
    return dics
# cls_data = read_jsonl(cls_file)
gen_data = read_jsonl(gen_file)
# random.shuffle(cls_data)
# random.shuffle(gen_data)

In [ ]:
msg_map = {}
cnt = 0
for dic in gen_data:
    msg = dic["msg"]
    if msg not in msg_map:
        msg_map[msg] = dic
    else:
        print(msg + "\n")
        print(msg_map[msg])
        print(dic)
        cnt += 1
        break
print(cnt)
print(len(gen_data))
print(cnt / len(gen_data))

In [13]:
i = 4
dic = cls_data[i]
open("patch.txt", "w").write(dic["patch"])
print(dic["y"])

1


In [ ]:
i = 1

In [24]:
i += 1
dic = gen_data[i]
open("patch.txt", "w").write(dic["patch"] + "\n\n\n" + dic["msg"])

7673

In [4]:
import json
path = "../../Processor/data/ref-valid.jsonl"
dics = []
with open(path, "r") as f:
    for line in f:
        js = json.loads(line)
        dics.append(js)

In [14]:
idx = 16
print(dics[idx]["comment"])
print(dics[idx]["hunk"])

All instances of calls to `Progress()` have a `1.0 / (float)numTasks` that should be changed to `1.0f / (float)numTasks)`
@@ -294,7 +294,7 @@ void DataConverter::_convertToOgr(const QStringList& inputs, const QString& outp
       convertOps.setProgress(
         Progress(
           ConfigOptions().getJobId(), JOB_SOURCE, Progress::JobState::Running,
-          (currentTask - 1) / numTasks, 1.0 / numTasks));
+          (currentTask - 1) / numTasks, 1.0f / numTasks));
       convertOps.apply(map);
       currentTask++;
       LOG_STATUS(

